Pipeline for Extracting Keywords from the dataset

In [86]:
import pandas as pd
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ethanvirtudazo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
nlp = spacy.load("en_core_web_sm")

path = "/Users/ethanvirtudazo/Desktop/DS105_Dataset/6622_jobs.xls"
df_import = pd.read_excel(path)

df_import.shape

(6623, 11)

In [48]:
df1 = df_import.iloc[:, 1:5]
df1.head()

df2 = df_import.iloc[:, [7,9,10]]
df2.head()

pdList = [df1,df2]
df = pd.concat(pdList,axis=1)
df.head()

df.shape

types = df.dtypes
print(types)

df_text_both = df.iloc[:, [0,1]]
df_text = df.iloc[:, [1]]

df_text_both.head()


title                  object
details                object
deadline       datetime64[ns]
opport_type            object
location               object
company                object
links                  object
dtype: object


,title,details
0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...
1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...
2,2023 HSBC Global Internship Programme (Hong Ko...,You’re excited about starting your career and ...
3,"Graduate Training Scheme, Capital Markets",Graduate Training Scheme – LondonGreySpark Par...
4,6-Months Internship – Sell-side Tech M&A,"At IPTP, we understand software from decades o..."


In [73]:
df_text.head()

,details
0,This London-based 6-month internship is an exc...
1,You’re excited about starting your career and ...
2,You’re excited about starting your career and ...
3,Graduate Training Scheme – LondonGreySpark Par...
4,"At IPTP, we understand software from decades o..."


In [104]:
ar_text = df_text.to_numpy()
ar_text = ar_text[0:101]

In [105]:
nlp = spacy.load("en_core_web_sm")

# Tokenize data using list comprehension
tokens = np.array([[token.text for token in nlp(sent[0])] for sent in ar_text])

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [106]:
tokens_filtered = tokens    

['Key',
 'ResponsibilitiesThis',
 'role',
 'reports',
 'directly',
 'to',
 'the',
 'MAM',
 'Section',
 'Head',
 'for',
 'Treasury',
 '/',
 'Linear',
 'Products',
 '.',
 'The',
 'MAM',
 'Department',
 'has',
 'two',
 'key',
 'responsibilities:1',
 ')',
 '     ',
 'Production',
 'and',
 'analysis',
 'of',
 'the',
 'P&L',
 'figures2',
 ')',
 '     ',
 'Production',
 'and',
 'analysis',
 'of',
 'the',
 'Market',
 'Risk',
 'indicators',
 'and',
 'monitoring',
 'of',
 'their',
 'associated',
 'limits',
 '.',
 'Systems',
 ':',
 'this',
 'production',
 'is',
 'carried',
 'out',
 'using',
 'Front',
 'Office',
 'systems',
 'and',
 'dedicated',
 'environments',
 '.',
 ' ',
 'Global',
 'View',
 'is',
 'the',
 'dedicated',
 'tool',
 'used',
 'for',
 'risk',
 'management',
 'purpose',
 '.',
 '·',
 '       ',
 'Collections',
 ',',
 'validation',
 'and',
 'control',
 'of',
 'market',
 'parameters',
 '.',
 '·',
 '       ',
 'Daily',
 'production',
 ',',
 'control',
 'and',
 'analysis',
 'of',
 'P&L',
 